In [1]:
import numpy as np

In [2]:
class Operation:
    """Represents a Node in the Computation Graph"""
    
    def __init__(self, input_nodes = []):
        """Constructs an Operation with input_nodes as inputs
           which computes outputs to zero or more consumers"""
        self.input_nodes = input_nodes
        self.consumers = []
        
        # Connect this node with its inputs, by adding it as a consumer to its inputs
        for input_node in self.input_nodes:
            input_node.consumers.append(self)
        
        # Add this operation to the Computation Graph
        # TODO: provide the graph explicitly
        _default_graph.operations.append(self)
    
    def compute(self):
        """Computes the output of the operation. Depends on the specific operation."""
        pass

In [3]:
class Add(Operation):
    def __init__(self, x, y):
        super().__init__(input_nodes=[x, y])
    
    def compute(self, x, y):
        return x + y

In [4]:
class Matmul(Operation):
    def __init__(self, A, B):
        super().__init__(input_nodes=[A, B])

    def compute(self, A, B):
        return A.dot(B)

In [5]:
class Placeholder:
    """Represents an input node which doesn't have any inputs
       and can only be consumed by other Nodes in the Computation Graph.
       
       The Placeholder has a fixed value. Acts like a constant."""
    
    def __init__(self):
        self.consumers = []
        
        # Register the placeholder in the Computation Graph
        # TODO: provide the graph explicitly
        _default_graph.placeholders.append(self)

In [6]:
class Variable:
    """Represents a parameter in the Computation Graph.
       This node doesn't have any inputs and has only consumers.
       
       The Variable's value can change. It is initialized to initial_value."""
    
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.consumers = []
        
        # Register the variable in the Computation Graph
        # TODO: provide the graph explicitly
        _default_graph.variables.append(self)

In [7]:
class Graph:
    """Represents the actual Computation Graph which has 3 types of Nodes:
       - placeholders
       - variables
       - operations
    """
    
    def __init__(self, placeholders=[], variables=[], operations=[]):
        self.placeholders = placeholders
        self.variables = variables
        self.operations = operations

    def as_default(self):
        global _default_graph
        _default_graph = self
        return _default_graph

In [8]:
class Session:
    """Represents a single execution of the whole Computation graph."""
    # TODO: provide the Graph explicitly
    
    def run(self, operation, feed_dict={}):
        """Performs a post-order traversal of all nodes in the Computation graph,
           so that all operations with known inputs are performed first.
        """
        
        nodes_in_post_order = Session.traverse_post_order(operation)
        
        outputs = {operation: None for operation in nodes_in_post_order}
        
        for node in nodes_in_post_order:
            if type(node) == Placeholder:
                outputs[node] = feed_dict[node]
            elif type(node) == Variable:
                outputs[node] = node.value
            elif isinstance(node, Operation):
                computed_inputs = [outputs[input_node] for input_node in node.input_nodes]
                outputs[node] = node.compute(*computed_inputs)

        return outputs[operation]

    @staticmethod
    def traverse_post_order(operation):
        operations_post_order = []
        
        def traverse(node):
            # Placeholders and Variables do not have input_nodes
            if isinstance(node, Operation):
                for input_node in node.input_nodes:
                    traverse(input_node)

            operations_post_order.append(node)
        
        traverse(operation)
        return operations_post_order

In [25]:
graph = Graph().as_default()

A = Variable(np.array([
    [1, 0],
    [0, -1]
]))
b = Variable(np.array([1, 1]))

x = Placeholder()

y = Add(Matmul(A, x), b)

Session().run(y, feed_dict={
    x: np.array([1, 2])
})

array([ 2, -1])